# Chapter 3: Streaming & Online Algorithms

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jmamath/interview_prep/blob/main/chapter_03_streaming_online_algorithms.ipynb)

## Introduction

In the real world of machine learning, data rarely arrives all at once in a neat package. Whether you're analyzing Twitter streams, processing IoT sensor data, or building recommendation systems that adapt to user behavior in real-time, you need algorithms that can handle massive, unbounded data streams.

This chapter introduces you to streaming and online algorithms—techniques that process data in a single pass without storing everything in memory. These algorithms are fundamental to modern ML systems, from real-time analytics pipelines to incremental model training.

You'll learn how to estimate frequencies with minimal memory, count unique items without storing them all, compute statistics on-the-fly, sample from infinite streams, and train models that learn incrementally as new data arrives.

## Learning Objectives
- Implement space-efficient frequency estimation with Count-Min Sketch
- Estimate cardinality with HyperLogLog
- Compute mean and variance in one pass using Welford's algorithm
- Sample uniformly from streams with Reservoir Sampling
- Train models incrementally with Online Gradient Descent

---

## Problem 1: Frequency Estimation with Count-Min Sketch (Easy-Medium)

### Contextual Introduction
Imagine you're analyzing search queries at Google. Billions of queries per day—you can't store them all. But you need to answer: "How many times was 'machine learning' searched?" Count-Min Sketch is a probabilistic data structure that estimates frequencies using constant memory, trading perfect accuracy for incredible space efficiency.

### Key Concepts
- **Probabilistic Data Structure**: Trades accuracy for space using randomness
- **Hash Functions**: Multiple independent hashes provide redundancy
- **Space-Time Tradeoff**: O(w*d) space, O(1) update and query time

### Problem Statement
Implement a Count-Min Sketch that estimates item frequencies in a data stream.

**Requirements**:
- Initialize with width and depth parameters
- Implement update() to process items
- Implement query() to estimate frequency
- Handle hash collisions gracefully

### Example: How Count-Min Sketch Works

```python
# Simplified example with width=4, depth=2
# Imagine we have 2 hash functions and 4 buckets per row

# Initial sketch (all zeros):
# Row 0: [0, 0, 0, 0]
# Row 1: [0, 0, 0, 0]

# Add "apple" (hash1("apple") % 4 = 2, hash2("apple") % 4 = 1)
# Row 0: [0, 0, 1, 0]  <- Incremented position 2
# Row 1: [0, 1, 0, 0]  <- Incremented position 1

# Add "banana" (hash1("banana") % 4 = 1, hash2("banana") % 4 = 3)
# Row 0: [0, 1, 1, 0]  <- Incremented position 1
# Row 1: [0, 1, 0, 1]  <- Incremented position 3

# Add "apple" again
# Row 0: [0, 1, 2, 0]  <- Incremented position 2 (now 2)
# Row 1: [0, 2, 0, 1]  <- Incremented position 1 (now 2)

# Query "apple": min(sketch[0,2], sketch[1,1]) = min(2, 2) = 2 ✓
# Query "banana": min(sketch[0,1], sketch[1,3]) = min(1, 1) = 1 ✓

# Why minimum? If there's a collision, one counter might be higher.
# The minimum gives the best estimate!
```


In [ ]:
import numpy as np
import hashlib

class CountMinSketch:
    def __init__(self, width: int = 1000, depth: int = 5):
        """
        Initialize Count-Min Sketch.
        
        Args:
            width: Number of columns (hash table size)
            depth: Number of rows (hash functions)
        """
        self.width = width
        self.depth = depth
        self.sketch = np.zeros((depth, width), dtype=int)
    
    def _hash(self, item: str, seed: int) -> int:
        """Hash an item to a column index."""
        h = hashlib.md5((str(item) + str(seed)).encode()).hexdigest()
        return int(h, 16) % self.width
    
    def update(self, item: str, count: int = 1):
        # TODO: For each row i from 0 to depth:
        #   - Compute hash index using _hash(item, i)
        #   - Increment sketch[i, index] by count
        pass
    
    def query(self, item: str) -> int:
        # TODO: Return the minimum count across all rows
        # This gives the best estimate of the item's frequency
        pass

def test_count_min_sketch():
    """Test Count-Min Sketch implementation."""
    sketch = CountMinSketch(width=100, depth=5)
    stream = ['apple', 'banana', 'apple', 'orange', 'apple', 'banana']
    for item in stream:
        sketch.update(item)
    
    # Test frequencies (using >= since it's an estimate)
    assert sketch.query('apple') >= 3, "Apple should appear at least 3 times"
    assert sketch.query('banana') >= 2, "Banana should appear at least 2 times"
    assert sketch.query('orange') >= 1, "Orange should appear at least 1 time"
    
    print("All Count-Min Sketch tests passed!")

# Uncomment to test
# test_count_min_sketch()


<details>
<summary>Click to reveal hint for Problem 1</summary>

**Hint**: For update(), loop through each row (0 to depth-1), hash the item with that row index as seed, and increment the cell. For query(), hash the item with each row index, get the count from each row, and return the minimum.

</details>


---


## Problem 2: Cardinality Estimation with HyperLogLog (Medium)

### Contextual Introduction
Facebook needs to count unique daily active users across billions of events. Storing every user ID would require terabytes. HyperLogLog estimates cardinality (number of unique items) using only kilobytes of memory, with typical error rates under 2%. It's used everywhere from database query optimizers to real-time analytics dashboards.

### Key Concepts
- **Cardinality**: The number of unique elements in a set
- **Probabilistic Counting**: Uses hash function properties to estimate cardinality
- **Harmonic Mean**: Combines estimates from multiple buckets for accuracy

### Problem Statement
Implement HyperLogLog to estimate the number of unique items in a stream with minimal memory.

**Requirements**:
- Initialize with precision parameter (number of buckets)
- Implement add() to process items
- Implement count() to estimate cardinality
- Achieve reasonable accuracy (within 10% error)

### Example: HyperLogLog Intuition

```python
# Concept: If you flip a coin until you get heads,
# the number of flips tells you about how many sequences you've seen

# Example with precision=2 (4 buckets: 00, 01, 10, 11)
# Hash "user_1" → binary: 10110101...
# First 2 bits (10) = bucket 2
# Remaining bits: count leading zeros + 1 = 3 (00110101... has 2 leading zeros)
# Register[2] = max(Register[2], 3) = 3

# Hash "user_2" → binary: 10010111...
# First 2 bits (10) = bucket 2 (same bucket!)
# Leading zeros + 1 = 2
# Register[2] = max(3, 2) = 3 (keep maximum)

# Hash "user_3" → binary: 01111001...
# First 2 bits (01) = bucket 1
# Leading zeros + 1 = 1
# Register[1] = 1

# Registers: [0, 1, 3, 0]

# Estimate cardinality:
# If we saw a run of 3 flips, we probably saw ~2^3 = 8 unique items
# But we have 4 buckets, so we use harmonic mean to combine estimates
# This gives a more accurate estimate

# Actual formula: alpha * m^2 / sum(2^(-register[i]))
# where m = number of buckets, alpha = bias correction
```


In [ ]:
import numpy as np
import hashlib

class HyperLogLog:
    def __init__(self, precision: int = 14):
        self.precision = precision
        self.m = 1 << precision  # 2^precision buckets
        self.registers = np.zeros(self.m, dtype=int)
    
    def _hash(self, item: str) -> int:
        return int(hashlib.md5(str(item).encode()).hexdigest(), 16)
    
    def add(self, item: str):
        # TODO: Hash the item, split into bucket index and remaining bits
        # Count leading zeros in remaining bits + 1
        # Update register[bucket] = max(register[bucket], leading_zeros)
        pass
    
    def count(self) -> float:
        # TODO: Compute harmonic mean of 2^registers
        # Return estimated cardinality
        pass

def test_hyperloglog():
    hll = HyperLogLog(precision=10)
    
    # Add unique items
    for i in range(10000):
        hll.add(f'user_{i}')
    
    estimate = hll.count()
    error = abs(estimate - 10000) / 10000
    
    print(f'Actual: 10000, Estimated: {estimate:.0f}, Error: {error:.2%}')
    assert error < 0.1, f'Error {error:.2%} exceeds 10%'
    print('All HyperLogLog tests passed!')

# Uncomment to test
# test_hyperloglog()

<details>
<summary>Click to reveal hint for Problem 2</summary>

**Hint**: Hash the item to get a 64-bit integer. Use the first `precision` bits to determine the bucket index. Count the leading zeros in the remaining bits (plus 1). Update the bucket's register to the maximum value seen. To estimate cardinality, use the formula: `alpha * m^2 / sum(2^(-register))` where alpha is a bias correction constant.

</details>

---

## Problem 3: Online Mean and Variance Computation (Medium)

### Contextual Introduction
Training neural networks requires computing statistics over mini-batches. Computing mean and variance naively can cause numerical instability. Welford's algorithm computes these statistics in one pass with numerical stability, used in BatchNorm layers and real-time monitoring systems.

### Key Concepts
- **Numerical Stability**: Avoiding catastrophic cancellation in floating-point arithmetic
- **Online Algorithm**: Updates statistics incrementally
- **Welford's Method**: Uses running mean to compute variance

### Problem Statement
Implement online mean and variance computation using Welford's algorithm.

**Requirements**:
- Maintain count, mean, and M2 (sum of squared differences)
- Implement update() to add new values
- Implement mean() and variance() methods
- Handle numerical stability

### Example: Welford's Algorithm Step-by-Step

```python
# Stream: [4, 7, 13, 16]
# Let's compute mean and variance online

# Initial state:
# n=0, mean=0, M2=0

# Add 4:
# n = 1
# delta = 4 - 0 = 4
# mean = 0 + 4/1 = 4
# delta2 = 4 - 4 = 0
# M2 = 0 + 4*0 = 0

# Add 7:
# n = 2
# delta = 7 - 4 = 3
# mean = 4 + 3/2 = 5.5
# delta2 = 7 - 5.5 = 1.5
# M2 = 0 + 3*1.5 = 4.5

# Add 13:
# n = 3
# delta = 13 - 5.5 = 7.5
# mean = 5.5 + 7.5/3 = 8
# delta2 = 13 - 8 = 5
# M2 = 4.5 + 7.5*5 = 42

# Add 16:
# n = 4
# delta = 16 - 8 = 8
# mean = 8 + 8/4 = 10
# delta2 = 16 - 10 = 6
# M2 = 42 + 8*6 = 90

# Final: mean = 10, variance = M2/(n-1) = 90/3 = 30 ✓

# Why stable? We never compute (x - mean)^2 directly for large sums,
# which can lose precision with floating-point arithmetic!
```

In [ ]:
class OnlineStats:
    def __init__(self):
        self.n = 0
        self._mean = 0.0
        self.M2 = 0.0
    
    def update(self, value: float):
        # TODO: Implement Welford's algorithm
        # Update count, mean, and M2
        pass
    
    def mean(self) -> float:
        return self._mean
    
    def variance(self) -> float:
        # TODO: Return M2 / n for population variance
        # Or M2 / (n-1) for sample variance
        pass

def test_online_stats():
    stats = OnlineStats()
    data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    
    for x in data:
        stats.update(x)
    
    expected_mean = 5.5
    expected_var = 8.25  # Sample variance
    
    assert abs(stats.mean() - expected_mean) < 0.01
    assert abs(stats.variance() - expected_var) < 0.01
    print('All Online Stats tests passed!')

# Uncomment to test
# test_online_stats()

<details>
<summary>Click to reveal hint for Problem 3</summary>

**Hint**: Welford's algorithm: `delta = value - mean`, `mean += delta / n`, `delta2 = value - mean`, `M2 += delta * delta2`. Variance is `M2 / (n-1)` for sample variance.

</details>

---

## Problem 4: Reservoir Sampling (Medium)

### Contextual Introduction
You're building a recommendation system that needs to sample from user activity streams. The stream is unbounded—you don't know when it ends. Reservoir sampling maintains a uniform random sample of k items from a stream of unknown length, ensuring each item has equal probability of being selected.

### Key Concepts
- **Uniform Random Sampling**: Each item has equal probability
- **Single-Pass Algorithm**: Process stream only once
- **Constant Memory**: Uses O(k) space regardless of stream length

### Problem Statement
Implement reservoir sampling to maintain k random samples from a stream.

**Requirements**:
- Initialize with sample size k
- Implement add() to process stream items
- Implement get_sample() to return current reservoir
- Ensure uniform distribution

### Example: Reservoir Sampling in Action

```python
# Goal: Keep k=3 random samples from a stream
# Stream: [1, 2, 3, 4, 5, 6, 7, 8]

# Step 1-3: Fill reservoir with first 3 items
# Reservoir: [1, 2, 3], n=3

# Step 4: Add item 4
# n=4, generate random j in [0, 4) → say j=2
# Since j < k (2 < 3), replace reservoir[2] = 4
# Reservoir: [1, 2, 4], n=4

# Step 5: Add item 5
# n=5, generate random j in [0, 5) → say j=4
# Since j >= k (4 >= 3), don't replace anything
# Reservoir: [1, 2, 4], n=5

# Step 6: Add item 6
# n=6, generate random j in [0, 6) → say j=1
# Since j < k (1 < 3), replace reservoir[1] = 6
# Reservoir: [1, 6, 4], n=6

# Step 7: Add item 7
# n=7, generate random j in [0, 7) → say j=5
# Since j >= k (5 >= 3), don't replace
# Reservoir: [1, 6, 4], n=7

# Step 8: Add item 8
# n=8, generate random j in [0, 8) → say j=0
# Since j < k (0 < 3), replace reservoir[0] = 8
# Reservoir: [8, 6, 4], n=8

# Each item had exactly 3/8 probability of being in final sample!
# Item i had probability: k/i * (i/(i+1)) * ((i+1)/(i+2)) * ... * ((n-1)/n) = k/n
```

In [ ]:
import random
from typing import List, Any

class ReservoirSampler:
    def __init__(self, k: int):
        self.k = k
        self.reservoir = []
        self.n = 0
    
    def add(self, item: Any):
        # TODO: Implement reservoir sampling algorithm
        # If reservoir not full, add item
        # Otherwise, randomly replace with probability k/n
        pass
    
    def get_sample(self) -> List[Any]:
        return self.reservoir.copy()

def test_reservoir_sampler():
    random.seed(42)
    sampler = ReservoirSampler(k=10)
    
    # Add 1000 items
    for i in range(1000):
        sampler.add(i)
    
    sample = sampler.get_sample()
    assert len(sample) == 10
    assert all(0 <= x < 1000 for x in sample)
    print('All Reservoir Sampler tests passed!')

# Uncomment to test
# test_reservoir_sampler()

<details>
<summary>Click to reveal hint for Problem 4</summary>

**Hint**: Fill reservoir with first k items. For item i > k, generate random number j in [0, i). If j < k, replace reservoir[j] with item i.

</details>

---

## Problem 5: Online Gradient Descent (Hard)

### Contextual Introduction
Modern ML systems need to adapt to new data continuously. Online gradient descent updates model parameters as new samples arrive, enabling real-time learning. It's used in online advertising, recommendation systems, and any application where the data distribution shifts over time.

### Key Concepts
- **Stochastic Gradient Descent**: Updates using single samples or mini-batches
- **Learning Rate Schedule**: Decreases learning rate over time for convergence
- **Regret**: Measures performance vs. optimal offline algorithm

### Problem Statement
Implement online gradient descent for linear regression that learns incrementally.

**Requirements**:
- Initialize weights and learning rate
- Implement partial_fit() for incremental updates
- Implement predict() method
- Use decreasing learning rate

### Example: Online Learning Step-by-Step

```python
# Problem: Learn y = 2x1 + 3x2 from streaming data
# Initial: weights = [0, 0], bias = 0, lr = 0.1

# Sample 1: x = [1, 1], y_true = 5
# Prediction: y_pred = [1, 1] @ [0, 0] + 0 = 0
# Error: error = 0 - 5 = -5
# Gradients: grad_w = [1, 1] * (-5) = [-5, -5]
#            grad_b = -5
# Update: weights = [0, 0] - 0.1 * [-5, -5] = [0.5, 0.5]
#         bias = 0 - 0.1 * (-5) = 0.5
# t = 1, next_lr = 0.1 / sqrt(2) ≈ 0.07

# Sample 2: x = [2, 0], y_true = 4
# Prediction: y_pred = [2, 0] @ [0.5, 0.5] + 0.5 = 1.5
# Error: error = 1.5 - 4 = -2.5
# Gradients: grad_w = [2, 0] * (-2.5) = [-5, 0]
#            grad_b = -2.5
# Update: weights = [0.5, 0.5] - 0.07 * [-5, 0] = [0.85, 0.5]
#         bias = 0.5 - 0.07 * (-2.5) = 0.675
# t = 2, next_lr = 0.1 / sqrt(3) ≈ 0.058

# ... continue for more samples ...
# Weights gradually converge to [2, 3]!

# Why decreasing learning rate?
# Early: Large steps to get close to optimum quickly
# Later: Small steps to fine-tune and converge
```

In [ ]:
import numpy as np

class OnlineLinearRegression:
    def __init__(self, n_features: int, learning_rate: float = 0.01):
        self.weights = np.zeros(n_features)
        self.bias = 0.0
        self.lr = learning_rate
        self.t = 0
    
    def partial_fit(self, X: np.ndarray, y: np.ndarray):
        # TODO: Compute prediction and error
        # Update weights using gradient descent
        # Use learning rate schedule: lr / sqrt(t+1)
        pass
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        # TODO: Return X @ weights + bias
        pass

def test_online_gradient_descent():
    # Generate simple linear data
    np.random.seed(42)
    X_train = np.random.randn(1000, 5)
    true_weights = np.array([1, 2, 3, 4, 5])
    y_train = X_train @ true_weights + np.random.randn(1000) * 0.1
    
    model = OnlineLinearRegression(n_features=5)
    
    # Train online
    for X, y in zip(X_train, y_train):
        model.partial_fit(X.reshape(1, -1), np.array([y]))
    
    # Test prediction
    X_test = np.random.randn(10, 5)
    y_test = X_test @ true_weights
    predictions = model.predict(X_test)
    
    mse = np.mean((predictions - y_test)**2)
    print(f'MSE: {mse:.4f}')
    assert mse < 1.0, f'MSE {mse:.4f} too high'
    print('All Online Gradient Descent tests passed!')

# Uncomment to test
# test_online_gradient_descent()

<details>
<summary>Click to reveal hint for Problem 5</summary>

**Hint**: For each sample, compute prediction `y_pred = X @ weights + bias`, error `error = y_pred - y`, gradient for weights `grad_w = X.T * error`, gradient for bias `grad_b = error`. Update: `weights -= lr * grad_w`, `bias -= lr * grad_b`. Use learning rate schedule `lr / sqrt(t+1)`.

</details>